In [2]:
# Seed value
# Apparently you may use different seed values at each stage
seed_value= 0

# 1. Set `PYTHONHASHSEED` environment variable at a fixed value
import os
os.environ['PYTHONHASHSEED']=str(seed_value)

# 2. Set `python` built-in pseudo-random generator at a fixed value
import random
random.seed(seed_value)

# 3. Set `numpy` pseudo-random generator at a fixed value
import numpy as np
np.random.seed(seed_value)

# 4. Set the `tensorflow` pseudo-random generator at a fixed value
import tensorflow as tf
tf.random.set_seed(seed_value)
# for later versions: 
tf.compat.v1.set_random_seed(seed_value)

# 5. Configure a new global `tensorflow` session

session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
tf.compat.v1.keras.backend.set_session(sess)

In [3]:
import os, shutil

path = '../data/big/benign'
for file in os.listdir(path):
    new = 'benign.' + file
    os.rename(os.path.join(path, file), os.path.join(path, new))

path = '../data/big/malignant'
for file in os.listdir(path):
    new = 'malignant.' + file
    os.rename(os.path.join(path, file), os.path.join(path, new))

In [4]:
# The path to the directory where the original
# dataset is uncompressed
original_dataset_dir = '../data/big'

# The directory where to
# store the smaller dataset
base_dir = '../data/idea1'
if not os.path.exists(base_dir):
    os.mkdir(base_dir)

# Create the directories for the 
# training partition
train_dir = os.path.join(base_dir, 'train')
if not os.path.exists(train_dir):
    os.mkdir(train_dir)
    
# validation partition
validation_dir = os.path.join(base_dir, 'validation')
if not os.path.exists(validation_dir):
    os.mkdir(validation_dir)

# test partition
test_dir = os.path.join(base_dir, 'test')
if not os.path.exists(test_dir):
    os.mkdir(test_dir)

print("train_dir=",train_dir)
print("validation_dir=",validation_dir)
print("test_dir=",test_dir)

train_dir= ../data/idea1\train
validation_dir= ../data/idea1\validation
test_dir= ../data/idea1\test


In [5]:
# Directory with the training benign pictures
train_benign_dir = os.path.join(train_dir, 'benign')
if not os.path.exists(train_benign_dir):
    os.mkdir(train_benign_dir)

# Directory with the training dog pictures
train_malignant_dir = os.path.join(train_dir, 'malignant')
if not os.path.exists(train_malignant_dir):
    os.mkdir(train_malignant_dir)

# Directory with the validation benign pictures
validation_benign_dir = os.path.join(validation_dir, 'benign')
if not os.path.exists(validation_benign_dir):
    os.mkdir(validation_benign_dir)

# Directory with the validation dog pictures
validation_malignant_dir = os.path.join(validation_dir, 'malignant')
if not os.path.exists(validation_malignant_dir):
    os.mkdir(validation_malignant_dir)

# Directory with the test benign pictures
test_benign_dir = os.path.join(test_dir, 'benign')
if not os.path.exists(test_benign_dir):
    os.mkdir(test_benign_dir)

# Directory with the test dog pictures
test_malignant_dir = os.path.join(test_dir, 'malignant')
if not os.path.exists(test_malignant_dir):
    os.mkdir(test_malignant_dir)

In [6]:
# Copy first 1000 benign images to train_benigns_dir
fnames = ['benign.{}.jpg'.format(i) for i in range(1,1000)]
for fname in fnames:
    src = original_dataset_dir + "/" + fname
    dst = train_benign_dir + "/" + fname
    shutil.copyfile(src, dst)

# Copy NEXT 250 benign images to validation_benigns_dir
fnames = ['benign.{}.jpg'.format(i) for i in range(1000, 1250)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(validation_benign_dir, fname)
    shutil.copyfile(src, dst)
    
# Copy NEXT 250 benign images to test_benign_dir
fnames = ['benign.{}.jpg'.format(i) for i in range(1250, 1500)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(test_benign_dir, fname)
    shutil.copyfile(src, dst)
    
# Copy first 1000 malignant images to train_malignant_dir
fnames = ['malignant.{}.jpg'.format(i) for i in range(1,1000)]
for fname in fnames:
    try: 
        src = os.path.join(original_dataset_dir, fname)
        dst = os.path.join(train_malignant_dir, fname)
        shutil.copyfile(src, dst)
    except:
        pass
    
# Copy next 500 malignant images to validation_malignant_dir
fnames = ['malignant.{}.jpg'.format(i) for i in range(1000, 1250)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(validation_malignant_dir, fname)
    shutil.copyfile(src, dst)
    
# Copy next 500 malignant images to test_malignant_dir
fnames = ['malignant.{}.jpg'.format(i) for i in range(1250, 1500)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(test_malignant_dir, fname)
    shutil.copyfile(src, dst)

In [7]:
# richtiges coding


from tensorflow.keras import layers
from tensorflow.keras import models

from tensorflow.keras import optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator


model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu',input_shape=(224, 244, 3)))

model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))


model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(learning_rate=1e-5),
              metrics=['acc'])



# All images will be [0,1] standardized
train_datagen = ImageDataGenerator(rescale=1.0/255)
valid_datagen = ImageDataGenerator(rescale=1.0/255)

training_generator = train_datagen.flow_from_directory(
        # This is the target directory
        train_dir,
        # All images will be resized to 150x150
        target_size=(224, 244),
        batch_size=20,
        # Since binary_crossentropy loss is used, binary labels are needed
        class_mode='binary')

validation_generator = valid_datagen.flow_from_directory(
        validation_dir,
        target_size=(224, 244),
        batch_size=20,
        class_mode='binary')

asdf = [1,1,1,1,1,1,1,1,1,1,1,1,1]
asdf[0:4]

print(type(training_generator))
# print(validation_generator[1])

Found 1995 images belonging to 2 classes.
Found 500 images belonging to 2 classes.
<class 'keras.preprocessing.image.DirectoryIterator'>


In [8]:
benign_input = "../data/only_pictures/benign"
malignant_input = "../data/only_pictures/malignant"

train_data_loc = "../data/only_pictures/benign"

benign_generator = train_datagen.flow_from_directory(
    # This is the target directory
    train_data_loc,
    # All images will be resized to 150x150
    target_size=(224, 224),
    # Since binary_crossentropy loss is used, binary labels are needed
    class_mode='binary')

malignant_generator = train_datagen.flow_from_directory(
    # This is the target directory
    malignant_input,
    # All images will be resized to 150x150
    target_size=(224, 224),
    # Since binary_crossentropy loss is used, binary labels are needed
    class_mode='binary')



Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.


In [9]:
train_dir = "../data/only_pictures"
training_generator = train_datagen.flow_from_directory(
    # This is the target directory
    train_dir,
    # All images will be resized to 150x150
    target_size=(224, 224),
    batch_size=1,
    # Since binary_crossentropy loss is used, binary labels are needed
    class_mode='binary',
    subset='training')


training_data = []

# malignant
max_value = (len(os.listdir(train_dir+"/benign")) + len(os.listdir(train_dir+"/malignant")))
print("the number of pictures in this dic is:",max_value)
for index, x in enumerate(training_generator):
    
    # safe label info as integer
    if(1 in x[1] ):
        temp_to_add = 1
    else:
        temp_to_add = 0

    # ingore the batch size and only take the first element of a list with one element
    for i in x[0]:
        training_data.append([i, temp_to_add])

    if(index % max_value == 0 and index != 0):
        break
    if(index % 100 == 0):
        print(index)
numpy_training_data = np.array(training_data)

Found 3297 images belonging to 2 classes.
the number of pictures in this dic is: 3297
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200


C:\Users\Bene\AppData\Local\Temp/ipykernel_17288/306576687.py:34: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  numpy_training_data = np.array(training_data)


In [10]:
len(training_data[3200])

# ebene 1 is the number of pictures 
# ebene 2 is malignant or benign
# ebene 3 is the batch size - this makes problems
# ebene 4 is the picture


2

In [11]:
import matplotlib.pyplot as plt

# plt.imshow(training_data[1997])

# shows if malignant or benign
# is one
print(training_data[111][1])
# is zero
print(training_data[0][1])


1
1


In [12]:
import pandas as pd

df = []

for i in training_data:
    temp_think = {}
    temp_think["label"] = i[1]
    temp_think["image"] = i[0]
    df.append(temp_think)
df = pd.DataFrame(df)

In [13]:
df.head()
# convert from list with 2 elements to dataframe

,label,image
0,1,"[[[0.45098042, 0.20392159, 0.08235294], [0.729..."
1,0,"[[[0.81568635, 0.74509805, 0.8470589], [0.8156..."
2,0,"[[[0.69411767, 0.5019608, 0.48627454], [0.6862..."
3,0,"[[[0.8352942, 0.5019608, 0.4901961], [0.839215..."
4,1,"[[[0.26666668, 0.227451, 0.27058825], [0.28627..."


In [15]:
from sklearn.model_selection import KFold

# Define the number of folds
k = 10

# Create a KFold object
kf = KFold(n_splits=k)
scores = []



# Loop over the folds
for train_index, test_index in kf.split(training_data):
    # Get the training and test sets for this fold
    
    temp_train = np.array(training_data)[train_index]

    temp_validate = np.array(training_data)[test_index]
    # y_train, y_test = training_data[train_index][0], training_data[test_index][0]
    
    # temp_train_dir_x = os.listdir(train_dir+"/benign")[train_index[0]:train_index[-1]]
    # temp_train_dir_x.extend(os.listdir(train_dir+"/malignant")[train_index[0]:train_index[-1]])
    # temp_validation_dir_x = os.listdir(validation_dir)[train_index[0]:train_index[-1]]
    # temp_validation_dir_x.extend(os.listdir(validation_dir)[train_index[0]:train_index[-1]])
   
    labels = np.array([x[1] for x in temp_train])
    images = np.array([x[0] for x in temp_train])
    print(len(labels[~np.isnan(labels)]), len(labels))
    print(len(images[~np.isnan(images)]), len(images))

    images = images[~np.isnan(images)]

    # Train the model on the training set
    model.fit(images ,labels , epochs=10, verbose=1, batch_size=20)

    # Evaluate the model on the test set
    score = model.score(temp_validate, verbose=0)

    # Append the score to a list
    scores.append(score)
    print(score)

# Calculate the mean score
mean_score = np.mean(scores)

C:\Users\Bene\AppData\Local\Temp/ipykernel_17288/414996032.py:16: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  temp_train = np.array(training_data)[train_index]
C:\Users\Bene\AppData\Local\Temp/ipykernel_17288/414996032.py:18: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  temp_validate = np.array(training_data)[test_index]


2968 2968
446767104 2968
Epoch 1/10


ValueError: in user code:

    File "c:\ProgramData\Anaconda3\lib\site-packages\keras\engine\training.py", line 1249, in train_function  *
        return step_function(self, iterator)
    File "c:\ProgramData\Anaconda3\lib\site-packages\keras\engine\training.py", line 1233, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\ProgramData\Anaconda3\lib\site-packages\keras\engine\training.py", line 1222, in run_step  **
        outputs = model.train_step(data)
    File "c:\ProgramData\Anaconda3\lib\site-packages\keras\engine\training.py", line 1023, in train_step
        y_pred = self(x, training=True)
    File "c:\ProgramData\Anaconda3\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "c:\ProgramData\Anaconda3\lib\site-packages\keras\engine\input_spec.py", line 295, in assert_input_compatibility
        raise ValueError(

    ValueError: Input 0 

In [ ]:
from sklearn.model_selection import KFold

# Define the number of folds
k = 10

# Create a KFold object
kf = KFold(n_splits=k)
scores = []



# Loop over the folds
for train_index, test_index in kf.split(training_data):
    # Get the training and test sets for this fold

    temp_train = np.array(training_data)[train_index]

    temp_validate = np.array(training_data)[test_index]

    labels = np.array([x[1] for x in temp_train])
    images = np.array([x[0] for x in temp_train])

    num_val_samples = len(labels)

    print('processing ' + str(i+1) + '.fold')

    # Prepare the validation data: data from partition # k
    val_data = train_data[i * num_val_samples: (i + 1) * num_val_samples]
    val_targets = train_targets[i * num_val_samples: (i + 1) * num_val_samples]

    # Prepare the training data: data from all other partitions
    partial_train_data = np.concatenate(
                        [train_data[:i * num_val_samples],
                         train_data[(i + 1) * num_val_samples:]],
                        axis=0)
    
    partial_train_targets = np.concatenate(
                        [train_targets[:i * num_val_samples],
                         train_targets[(i + 1) * num_val_samples:]],
                        axis=0)

    # Build the model (each model should be build with the same parameters)
    
    # Train the model (in silent mode, verbose=0)
    '''Write everything to the history object and explore it afterwards (see below)'''
    history = model.fit(partial_train_data, partial_train_targets,
                        validation_data=(val_data, val_targets),
                        epochs=num_epochs, batch_size=1, verbose=0)

    # Evaluate the model on the test set
    score = model.score(temp_validate, verbose=0)

    # Append the score to a list
    scores.append(score)
    print(score)

# Calculate the mean score
mean_score = np.mean(scores)